# First Howemork. Optimization and Analytics

#### Done by Javier Alzuaz & Jaime Lobato

### Motivation for the Optimization problem: improving the Indiana Pacers’ performance through optimal shot allocation and lineup selection

The Indiana Pacers reached the NBA Finals last season, establishing themselves as one of the strongest teams in the league. However, their performance this year has significantly got worse, mainly due to a combination of injuries, inconsistent rotations, and the need to rely more heavily on G-League call-ups to maintain roster depth.

This sudden transition from contending for a championship to being in the last place in the NBA standings creates a realistic decision-making scenario for applying optimization techniques.

The coaching staff faces some important questions:

- Given the current roster situation, how should the team allocate its offensive shots among the available players to maximize expected scoring?

- Given the current roster performances, injuries, and the mix of NBA and G-League players, what starting lineup would maximize the team’s on-court effectiveness?

This is a strategic problem with limited resources (healthy players, budget) and clear performance metrics, making it ideal for formulation as a Linear Optimization and later a Mixed-Integer Optimization model.

## Part a. Linear Optimization Model - Shot Allocation

### Mathematical Formulation of the Problem

### Sets

  - $P$: set of all players available.
  - $T = \{2P, 3P\}$: set of shot types.

Position subsets:
  - $P^{PG} \subseteq P$ represents the set of point guards (PGs).
  - $P^{SG} \subseteq P$ represents the set of shooting guards (SGs).
  - $P^{F} \subseteq P$ represents the set of forwards (SFs or PFs).
  - $P^{C} \subseteq P$ represents the set of centers (Cs).

### Parameters

For each player $i \in P$ and $t \in T$:

- $e_{i, t}$: expected points per shot of type $t$ and player $i$.

For example: $e_{i, 2P} = 2 \cdot 2P\%_{i}, \quad e_{i, 3P} = 3 \cdot 3P\%_{i}$

- $S_{i, 2}^{\max}: \text{ max 2-point shots for player } i$ 
- $S_{i, 3}^{\max}: \text{ max 3-point shots for player } i$


With this, we ensure that the shot selection is distributed along the entire roster.

Team parameter:

- $S^{team}$: total number of shots attempts taken by the team in a game.

### Decision Variables

$$s_{i, t} \geq 0$$

- $s_{i, t}$: number of shots of type $t$ each player $i$ takes.


### Objective Function

We want to maximize the expected total points.

\begin{alignat*}{2}
\max \quad & \sum_{i \in P} \sum_{t \in T} e_{i,t} s_{i,t} \\
\text{s.t.} \quad & \sum_{i \in P} \sum_{t \in T} s_{i,t} = S^{\text{team}} & \quad & \text{(total shots)} \\
& \sum_{t \in T} s_{i,2} \leq S_{i,2}^{\max} & \quad & \forall i \in P \quad \text{(player max 2-point shots)} \\
& \sum_{t \in T} s_{i,3} \leq S_{i,3}^{\max} & \quad & \forall i \in P \quad \text{(player max 3-point shots)} \\
& s_{i,t} \geq 0 & \quad & \forall i \in P, t \in T \quad \text{(non-negativity)}
\end{alignat*}


In [2]:
import pandas as pd
from pyomo.environ import *
import numpy as np

data = pd.read_csv("pacers_stats.csv" ,sep =";", encoding='latin-1')

# We might also need per game stats for making some calculations easier
data_per_game = pd.read_csv("pacers_stats_per_game.csv" ,sep =";", encoding='latin-1')

data_per_game

,ï»¿Rk,Player,Age,Pos,G,GS,MP,FG,FGA,FG%,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Awards,Player-additional
0,1.0,Bennedict Mathurin,23.0,SF,2,2,36.5,8.5,15.5,0.548,...,5.5,7.0,2.5,0.0,0.0,2.5,3.0,31.0,NaN,mathube01
1,2.0,Pascal Siakam,31.0,PF,11,11,34.9,8.6,19.4,0.446,...,5.7,7.4,5.3,1.1,0.4,2.6,3.3,24.1,NaN,siakapa01
2,3.0,Aaron Nesmith,26.0,SF,11,11,30.5,4.9,13.4,0.367,...,2.9,4.5,1.5,0.8,0.3,0.7,2.5,15.5,NaN,nesmiaa01
3,4.0,Jarace Walker,22.0,PF,12,7,29.1,3.4,11.5,0.297,...,4.4,5.2,3.3,0.4,0.6,2.1,2.1,10.3,NaN,walkeja02
4,5.0,Andrew Nembhard,26.0,PG,5,5,27.4,5.2,14.2,0.366,...,1.2,1.4,6.8,0.4,0.0,2.2,2.6,17.2,NaN,nembhan01
5,6.0,Obi Toppin,27.0,PF,3,0,27.3,5.0,12.0,0.417,...,5.7,6.7,1.7,1.0,0.0,2.0,2.3,14.0,NaN,toppiob01
6,7.0,Ben Sheppard,24.0,SG,12,5,25.4,2.5,7.8,0.323,...,3.8,4.7,1.7,0.4,0.2,0.7,2.6,6.8,NaN,sheppbe01
7,8.0,Quenton Jackson,27.0,PG,5,3,20.2,4.0,7.4,0.541,...,2.4,3.4,3.6,1.0,0.2,1.2,2.0,11.8,NaN,jacksqu01
8,9.0,James Wiseman,24.0,C,1,1,20.0,2.0,3.0,0.667,...,0.0,4.0,0.0,0.0,1.0,3.0,2.0,4.0,NaN,wisemja01
9,10.0,Jeremiah Robinson-Earl,25.0,PF,7,2,19.4,2.0,5.6,0.359,...,3.4,6.4,1.0,0.7,0.0,0.3,1.1,5.4,NaN,robinje02


We create parameters for the model

In [3]:
# Index for players
players_idx = list(data_per_game.index)

# Shot types
shot_types = ["2P", "3P"]

# Build expected points per shot: e_{i,2P} = 2 * 2P%, e_{i,3P} = 3 * 3P%
e = {}
for i in players_idx:
    two_p_pct = data_per_game.loc[i, "2P%"]
    three_p_pct = data_per_game.loc[i, "3P%"]
    if pd.isna(three_p_pct):
        three_p_pct = 0.0
    e[(i, "2P")] = 2 * two_p_pct
    e[(i, "3P")] = 3 * three_p_pct


""" Max shots per player: 1.25 * their FGA (from the stats file). We use 1.25 as a scaling factor to allow for some flexibility, 
but we want to limit it based on their usual attempts """

S2_max = {}
S3_max = {}

for i in players_idx:
    two_pa = data_per_game.loc[i, "2PA"]   # average 2-point attempts per game
    three_pa = data_per_game.loc[i, "3PA"] # average 3-point attempts per game (may be NaN)

    # Max 2P = 1.25 * 2PA
    S2_max[i] = 1.25 * two_pa

    # Max 3P = 1.25 * 3PA
    S3_max[i] = 1.25 * three_pa


# Total team shots in a game. According to the stats file, the Pacers attempt around 95 shots per game.
S_team = 95


We build the model

In [4]:
from pyomo.environ import *

from itertools import combinations  

model = ConcreteModel()

# Sets
model.PLAYERS = Set(initialize=players_idx)
model.TYPES = Set(initialize=shot_types)

# Parameters
model.e = Param(model.PLAYERS, model.TYPES, initialize=e)
model.S2_max = Param(model.PLAYERS, initialize=S2_max)
model.S3_max = Param(model.PLAYERS, initialize=S3_max)
model.S_team = Param(initialize=S_team)

# Decision variables: s_{i,t} >= 0
model.s = Var(model.PLAYERS, model.TYPES, domain=NonNegativeReals)


In [5]:
# We define the function to maximize: total expected points

def objective_rule(m):
    return sum(m.e[i, t] * m.s[i, t] for i in m.PLAYERS for t in m.TYPES)

model.OBJ = Objective(rule=objective_rule, sense=maximize)


In [9]:
# We define the constraints

# 1. Total shots constraint
def total_shots_rule(m):
    return sum(m.s[i, t] for i in m.PLAYERS for t in m.TYPES) == m.S_team

model.TotalShots = Constraint(rule=total_shots_rule)

# Max 2P attempts per player
def max_2p_rule(m, i):
    return m.s[i, "2P"] <= m.S2_max[i]

model.Max2PShots = Constraint(model.PLAYERS, rule=max_2p_rule)

# Max 3P attempts per player
def max_3p_rule(m, i):
    return m.s[i, "3P"] <= m.S3_max[i]

model.Max3PShots = Constraint(model.PLAYERS, rule=max_3p_rule)


model.dual = Suffix(direction=Suffix.IMPORT_EXPORT)



'pyomo.core.base.constraint.ScalarConstraint'>) on block unknown with a new
Component (type=<class
'pyomo.core.base.constraint.AbstractScalarConstraint'>). This is usually
indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().
'pyomo.core.base.constraint.IndexedConstraint'>) on block unknown with a new
Component (type=<class 'pyomo.core.base.constraint.IndexedConstraint'>). This
is usually indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().
'pyomo.core.base.constraint.IndexedConstraint'>) on block unknown with a new
Component (type=<class 'pyomo.core.base.constraint.IndexedConstraint'>). This
is usually indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().


In [11]:
# We solve the model

solver = SolverFactory("glpk")  # or "cbc", "gurobi", etc.

result = solver.solve(model, tee=True)

print(result.solver.termination_condition)


GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write C:\Users\ralzu\AppData\Local\Temp\tmp_lkho1ey.glpk.raw --wglp C:\Users\ralzu\AppData\Local\Temp\tmpapp220g6.glpk.glp
 --cpxlp C:\Users\ralzu\AppData\Local\Temp\tmppycz5_ez.pyomo.lp
Reading problem data from 'C:\Users\ralzu\AppData\Local\Temp\tmppycz5_ez.pyomo.lp'...
43 rows, 42 columns, 84 non-zeros
301 lines were read
Writing problem data to 'C:\Users\ralzu\AppData\Local\Temp\tmpapp220g6.glpk.glp'...
252 lines were written
GLPK Simplex Optimizer 5.0
43 rows, 42 columns, 84 non-zeros
Preprocessing...
1 row, 39 columns, 39 non-zeros
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 1
      0: obj =   1.084900000e+02 inf =   8.188e+01 (1)
     10: obj =   9.237887500e+01 inf =   0.000e+00 (0)
*    24: obj =   1.108421250e+02 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Time used:  

In [12]:
solution_rows = []

for i in model.PLAYERS:
    s_2p = value(model.s[i, "2P"])
    s_3p = value(model.s[i, "3P"])
    total_shots_i = s_2p + s_3p
    contrib = value(model.e[i, "2P"]) * s_2p + value(model.e[i, "3P"]) * s_3p
    
    # Only include players who actually shoot
    if total_shots_i > 1e-6:
        solution_rows.append({
            "Player": data.loc[i, "Player"] if "Player" in data.columns else i,
            "2P_shots": s_2p,
            "3P_shots": s_3p,
            "Total_shots": total_shots_i,
            "Expected_points": contrib
        })

solution_df = pd.DataFrame(solution_rows)
solution_df = solution_df.sort_values(by="Expected_points", ascending=False)

print("\nOptimal shot allocation:")
print(solution_df)

total_expected_points = solution_df["Expected_points"].sum()
print(f"\nTotal expected points: {total_expected_points:.2f}")
print(f"Total shots used: {solution_df['Total_shots'].sum():.0f} (should be {S_team})")



Optimal shot allocation:
                Player  2P_shots  3P_shots  Total_shots  Expected_points
0   Bennedict Mathurin    13.125     6.250       19.375        24.363750
1        Pascal Siakam    18.375     5.750       24.125        23.421000
4      Quenton Jackson     6.250     3.000        9.250        11.500000
2        Aaron Nesmith     0.000     9.375        9.375        10.490625
3           Obi Toppin     7.875     0.000        7.875         9.954000
6       Isaiah Jackson     7.500     0.000        7.500         8.340000
9         Tony Bradley     4.750     0.000        4.750         6.194000
5        James Wiseman     3.750     0.000        3.750         5.002500
11         Mac McClung     3.375     0.000        3.375         4.218750
10       Johnny Furphy     0.000     1.250        1.250         2.501250
8             Jay Huff     1.875     0.000        1.875         2.501250
7          RayJ Dennis     0.000     2.500        2.500         2.355000

Total expected points: 1

## Part c. Sensitivities associated with each constraint

In [13]:
print("\nDual values (shadow prices) for all active constraints:\n")

for c in model.component_objects(Constraint, active=True):
    print(f"Constraint block: {c.name}")
    for index in c:
        constr = c[index]
        dual_val = model.dual.get(constr, 0.0)
        print(f"  {constr.name}: dual = {dual_val:.4f}")
    print()


Dual values (shadow prices) for all active constraints:

Constraint block: TotalShots
  TotalShots: dual = 0.9420

Constraint block: Max2PShots
  Max2PShots[0]: dual = 0.2000
  Max2PShots[1]: dual = 0.0200
  Max2PShots[2]: dual = 0.0000
  Max2PShots[3]: dual = 0.0000
  Max2PShots[4]: dual = 0.0000
  Max2PShots[5]: dual = 0.3220
  Max2PShots[6]: dual = 0.0000
  Max2PShots[7]: dual = 0.1780
  Max2PShots[8]: dual = 0.3920
  Max2PShots[9]: dual = 0.0000
  Max2PShots[10]: dual = 0.1700
  Max2PShots[11]: dual = 0.0000
  Max2PShots[12]: dual = 0.3920
  Max2PShots[13]: dual = 0.0000
  Max2PShots[14]: dual = 0.3620
  Max2PShots[15]: dual = 0.0000
  Max2PShots[16]: dual = 0.0000
  Max2PShots[17]: dual = 0.0000
  Max2PShots[18]: dual = 0.3080
  Max2PShots[19]: dual = 0.0000
  Max2PShots[20]: dual = 0.0000

Constraint block: Max3PShots
  Max3PShots[0]: dual = 0.5580
  Max3PShots[1]: dual = 0.0570
  Max3PShots[2]: dual = 0.1770
  Max3PShots[3]: dual = 0.0000
  Max3PShots[4]: dual = 0.0000
  Max3PS

### Interpretation of Sensitivity Results

In a linear optimization model, each constraint has an associated dual value that measures how the objective would change if the constraint was altered i.e. the effect of the constraints in the optimal objective.

- A non-zero dual value indicates that the corresponding variable is not a 'binding' constraint, which indicates that it has a direct effect on the solution and is 'active' in the model. In our context, allowing one additional shot would increase the team’s expected points by approximately the value of the dual

- Conversely, a zero dual value means the constraint is 'non-binding': the player is not using up their full shooting capacity in the optimal solution, so increasing their shot limit would not change the objective. Zero duals often occur for less efficient shooters, role players, or players whose shot type (2P or 3P) is not used by the optimizer.

In summary, non-zero dual values identify the constraints that impact team’s performance, while zero dual values correspond to constraints that are irrelevant to the model.

#### Total Shots Constraint:

This constraint has a dual value of 0.942, which represents the marginal value of one additional team shot within the current optimal solution. In other words, if the Pacers shot one additional shot, the expected points would increase by approximately 0.942 points.

This value is binding constant: the team is using up all available shot attempts and having more would improve performance.


#### 2-point Shots Constraint:

The dual value associated with each player’s 2-point constraint tells us how valuable it would be to allow that player to take one additional 2-point attempt.

We can see that there are several players with a zero dual value, which are Aaron Nesmith, Jarace Walker, Andrew Nembhard, Ben Sheppard, Jeremiah Robinson-Earl, RayJ Dennis, Monte Morris, Cody Martin, T.J. McConnell, Johnny Furphy and Taelon Peter. These are players to which the optimizer does not assign all their allowed shots because doing so would not increase the expected points. This may be due to the players being less efficient scorers or low-usage offensive players.

The rest of players have positive dual value, meaning that the player is already taking as many 2-pointers as allowed, and giving them one more shot would increase the team's expected points by approximately the dual value.


#### 3-point Shots Constraint:

The interpretation is equivalent to the 2-pointers one.

We can see that only five players would increase the team's expected points by taking more threes. These players are Bennedict Mathurin, Pascal Siakam, Aaron Nesmith, Quenton Jackson and Johnny Furphy.


## Part d. Mixed-Integer Optimization Problem - Optimal Lineup 

### Mathematical Formulation of the Problem

### Sets

  - $P$: set of all players available.

Position subsets:
  - $P^{PG} \subseteq P$ represents the set of point guards (PGs).
  - $P^{SG} \subseteq P$ represents the set of shooting guards (SGs).
  - $P^{F} \subseteq P$ represents the set of forwards (SFs or PFs).
  - $P^{C} \subseteq P$ represents the set of centers (Cs).

### Parameters

For each player $i \in P$:

- $v_{i}$: valuation (performance score) for each player $i$.

We are going to compute this as follows:

- $v_{i} = \mathbf{FG\%}_{i} + \mathbf{FT\%}_{i} + \mathbf{3P\%}_{i} + \mathbf{PTS}_{i} + \mathbf{STL}_{i} + \mathbf{TRB}_{i} + \mathbf{AST}_{i} + \mathbf{BLK}_{i} - \mathbf{TOV}_{i} - \mathbf{PF}_{i}$

We define a linear performance index that rewards scoring efficiency (shooting percentages), points, rebounds, assists, steals and blocks, and penalizes negative aspects of the game as turnovers and personal fouls.

### Decision Variables

- $x_{i} = 
\begin{cases}
    1, & \text{if player } i \text{ is selected in the starting lineup,} \\
    0, & \text{otherwise.}
\end{cases}$

  $x_{i} \in \{0, 1\} \quad \forall i \in P$.


### Objective Function

We want to maximize the total valuation of the selected lineup.

\begin{alignat*}{2}
\max \quad & \sum_{i \in P} v_{i} x_{i} \\
\text{s.t.} \quad & \sum_{i \in P} x_{i} = 5 & \quad & \text{(lineup size)} \\
& \sum_{i \in P^{\text{PG}}} x_{i} \geq 1 & \quad & \text{(at least 1 PG)} \\
& \sum_{i \in P^{\text{SG}}} x_{i} \geq 1 & \quad & \text{(at least 1 SG)} \\
& \sum_{i \in P^{\text{C}}} x_{i} \geq 1 & \quad & \text{(at least 1 C)} \\
& \sum_{i \in P^{\text{F}}} x_{i} \geq 2 & \quad & \text{(at least 2 forwards)} \\
& x_{i} \in \{0, 1\} \quad \forall i \in P & \quad & \text{(binary decisions)}
\end{alignat*}


In [1]:
import pandas as pd
from pyomo.environ import *
import numpy as np

In [2]:
data = pd.read_csv("pacers_stats.csv" ,sep =";", encoding='latin-1')

data

,Rk,Player,Age,Pos,G,GS,MP,FG,FGA,FG%,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Awards,Player-additional
0,1.0,Bennedict Mathurin,23.0,SF,2,2,365,85,155,0.548,...,55,70,25,0,0,25,30,310,NaN,mathube01
1,2.0,Pascal Siakam,31.0,PF,11,11,349,86,194,0.446,...,57,74,53,11,4,26,33,241,NaN,siakapa01
2,3.0,Aaron Nesmith,26.0,SF,11,11,305,49,134,0.367,...,29,45,15,8,3,7,25,155,NaN,nesmiaa01
3,4.0,Jarace Walker,22.0,PF,12,7,291,34,115,0.297,...,44,52,33,4,6,21,21,103,NaN,walkeja02
4,5.0,Andrew Nembhard,26.0,PG,5,5,274,52,142,0.366,...,12,14,68,4,0,22,26,172,NaN,nembhan01
5,6.0,Obi Toppin,27.0,PF,3,0,273,50,120,0.417,...,57,67,17,10,0,20,23,140,NaN,toppiob01
6,7.0,Ben Sheppard,24.0,SG,12,5,254,25,78,0.323,...,38,47,17,4,2,7,26,68,NaN,sheppbe01
7,8.0,Quenton Jackson,27.0,PG,5,3,202,40,74,0.541,...,24,34,36,10,2,12,20,118,NaN,jacksqu01
8,9.0,James Wiseman,24.0,C,1,1,200,20,30,0.667,...,0,40,0,0,10,30,20,40,NaN,wisemja01
9,10.0,Jeremiah Robinson-Earl,25.0,PF,7,2,194,20,56,0.359,...,34,64,10,7,0,3,11,54,NaN,robinje02


In [3]:
data["Pos"] = data["Pos"].fillna("").astype(str)
data['valoration']= data['FG%']+data['FT%']+data['3P%']+data['PTS']+data['STL']+data['TRB']+data['AST']+data['BLK']-data['TOV']-data['PF']
data

,Rk,Player,Age,Pos,G,GS,MP,FG,FGA,FG%,...,TRB,AST,STL,BLK,TOV,PF,PTS,Awards,Player-additional,valoration
0,1.0,Bennedict Mathurin,23.0,SF,2,2,365,85,155,0.548,...,70,25,0,0,25,30,310,NaN,mathube01,351.933
1,2.0,Pascal Siakam,31.0,PF,11,11,349,86,194,0.446,...,74,53,11,4,26,33,241,NaN,siakapa01,325.423
2,3.0,Aaron Nesmith,26.0,SF,11,11,305,49,134,0.367,...,45,15,8,3,7,25,155,NaN,nesmiaa01,195.535
3,4.0,Jarace Walker,22.0,PF,12,7,291,34,115,0.297,...,52,33,4,6,21,21,103,NaN,walkeja02,157.402
4,5.0,Andrew Nembhard,26.0,PG,5,5,274,52,142,0.366,...,14,68,4,0,22,26,172,NaN,nembhan01,211.579
5,6.0,Obi Toppin,27.0,PF,3,0,273,50,120,0.417,...,67,17,10,0,20,23,140,NaN,toppiob01,192.593
6,7.0,Ben Sheppard,24.0,SG,12,5,254,25,78,0.323,...,47,17,4,2,7,26,68,NaN,sheppbe01,106.327
7,8.0,Quenton Jackson,27.0,PG,5,3,202,40,74,0.541,...,34,36,10,2,12,20,118,NaN,jacksqu01,169.763
8,9.0,James Wiseman,24.0,C,1,1,200,20,30,0.667,...,40,0,0,10,30,20,40,NaN,wisemja01,NaN
9,10.0,Jeremiah Robinson-Earl,25.0,PF,7,2,194,20,56,0.359,...,64,10,7,0,3,11,54,NaN,robinje02,122.430


In [4]:
players_idx = list(data.index)

PG_idx = [i for i in players_idx if "PG" in data.loc[i, "Pos"]]
SG_idx = [i for i in players_idx if "SG" in data.loc[i, "Pos"]]
C_idx  = [i for i in players_idx if data.loc[i, "Pos"] == "C" or " C" in data.loc[i,"Pos"]]
F_idx  = [i for i in players_idx if any(p in data.loc[i, "Pos"] for p in ["SF", "PF"])]

# Diccionario de valoraciones
valoration_dict = data["valoration"].to_dict()

In [5]:
from pyomo.environ import *

import pandas as pd
from pyomo.environ import ConcreteModel, Var, Objective, Constraint, SolverFactory, NonNegativeReals, maximize
from itertools import combinations  
model = ConcreteModel()
model.PLAYERS = Set(initialize=players_idx)

model.valoration = Param(
    model.PLAYERS,
    initialize=valoration_dict
)

model.x = Var(model.PLAYERS, domain=Binary)

# Conjunto de jugadores
"""model.PLAYERS = Set(initialize=players_idx)

# Parámetro: valoración de cada jugador
model.valoration = Param(
    model.PLAYERS,
    initialize=valoration_dict,
    within=float
)"""

"""# Variable binaria: x[i] = 1 si el jugador i está en el quinteto
model.x = Var(model.PLAYERS, domain=Binary)"""


'Any'. The default domain for Param objects is 'Any'.  However, we will be
changing that default to 'Reals' in the future.  If you really intend the
domain of this Paramto be 'Any', you can suppress this warning by explicitly
specifying 'within=Any' to the Param constructor.  (deprecated in 5.6.9, will
be removed in (or after) 6.0) (called from
c:\Users\ralzu\anaconda3\envs\Optimization_Analytics\Lib\site-
packages\pyomo\core\base\indexed_component.py:718)


'# Variable binaria: x[i] = 1 si el jugador i está en el quinteto\nmodel.x = Var(model.PLAYERS, domain=Binary)'

In [6]:
def objective_rule(m):
    return sum(m.valoration[i] * m.x[i] for i in m.PLAYERS)

model.OBJ = Objective(rule=objective_rule, sense=maximize)
def lineup_size_rule(m):
    return sum(m.x[i] for i in m.PLAYERS) == 5

model.LineupSize = Constraint(rule=lineup_size_rule)

# 5.2. Al menos 1 base (PG)
def pg_rule(m):
    return sum(m.x[i] for i in PG_idx) >= 1

model.MinPG = Constraint(rule=pg_rule)

# 5.3. Al menos 1 escolta (SG)
def sg_rule(m):
    return sum(m.x[i] for i in SG_idx) >= 1

model.MinSG = Constraint(rule=sg_rule)

# 5.4. Al menos 1 pívot (C)
def c_rule(m):
    return sum(m.x[i] for i in C_idx) >= 1

model.MinC = Constraint(rule=c_rule)

# 5.5. Al menos 2 aleros/interiores (SF o PF)
def f_rule(m):
    return sum(m.x[i] for i in F_idx) >= 2

model.MinF = Constraint(rule=f_rule)


In [10]:
possible_solvers = ["glpk", "cbc", "gurobi", "cplex"]
solver = None
solver_name = None

for name in possible_solvers:
    try:
        if name == "glpk":
            s = SolverFactory(name, io="lp")  # 👈 force GLPK native LP format
        else:
            s = SolverFactory(name)
        if s is not None and s.available():
            solver = s
            solver_name = name
            break
    except:
        continue

use_pyomo_solution = False
results = None

if solver is None:
    print("\n⚠️  No se encontró ningún solver MILP instalado (glpk, cbc, gurobi, cplex).")
else:
    print(f"\nUsando solver: {solver_name}")
    try:
        results = solver.solve(model, tee=False)
        tc = results.solver.termination_condition
        print("Termination condition:", tc)

        if tc in [TerminationCondition.optimal, TerminationCondition.feasible]:
            use_pyomo_solution = True
        else:
            print(f"⚠️ Solver {solver_name} no terminó de forma óptima/viable. Se usará búsqueda por fuerza bruta.")
    except Exception as e:
        print(f"⚠️ Error al ejecutar el solver {solver_name}: {e}")
        print("Se usará búsqueda por fuerza bruta.")



Usando solver: glpk
ERROR: Solver (glpk) returned non-zero return code (1)
ERROR: Solver log: GLPSOL--GLPK LP/MIP Solver 5.0 Parameter(s) specified in
the command line:
     --write C:\Users\ralzu\AppData\Local\Temp\tmp8iqiejdy.glpk.raw --wglp
     C:\Users\ralzu\AppData\Local\Temp\tmpmbip1v33.glpk.glp --cpxlp
     C:\Users\ralzu\AppData\Local\Temp\tmpktyuc9k8.pyomo.lp
    Reading problem data from
    'C:\Users\ralzu\AppData\Local\Temp\tmpktyuc9k8.pyomo.lp'...
    C:\Users\ralzu\AppData\Local\Temp\tmpktyuc9k8.pyomo.lp:13: constraints
    section missing CPLEX LP file processing error
⚠️ Error al ejecutar el solver glpk: Solver (glpk) did not exit normally
Se usará búsqueda por fuerza bruta.


In [8]:
if use_pyomo_solution:
    selected_idx = [i for i in model.PLAYERS if value(model.x[i]) > 0.5]
    method_used = "Pyomo (MILP)"
else:
    print("\n▶ Ejecutando búsqueda por fuerza bruta en todas las combinaciones de 5 jugadores...")
    best_score = -np.inf
    best_combo = None

    for combo in combinations(players_idx, 5):
        subset = data.loc[list(combo)]
        positions = subset["Pos"]

        n_pg = (positions.str.contains("PG")).sum()
        n_sg = (positions.str.contains("SG")).sum()
        n_c  = (positions.str.contains("C")).sum()
        n_f  = (positions.str.contains("SF") | positions.str.contains("PF")).sum()

        if not (n_pg >= 1 and n_sg >= 1 and n_c >= 1 and n_f >= 2):
            continue

        total_val = subset["valoration"].sum()
        if total_val > best_score:
            best_score = total_val
            best_combo = combo

    selected_idx = list(best_combo)
    method_used = "búsqueda exhaustiva en Python"

# ---------------------------------------------------------
# 7. Mostrar quinteto óptimo
# ---------------------------------------------------------
best_lineup = data.loc[selected_idx].copy()
total_val = best_lineup["valoration"].sum()

print(f"\nMejor quinteto encontrado usando {method_used}")
print(f"Valoration total = {total_val:.2f}\n")

print(
    best_lineup[
        ["Player", "Pos", "PTS", "TRB", "AST", "STL", "BLK", "TOV", "PF", "valoration"]
    ].to_string(index=False)
)

print("\nDistribución de posiciones en el quinteto:")
print(best_lineup["Pos"].value_counts())


▶ Ejecutando búsqueda por fuerza bruta en todas las combinaciones de 5 jugadores...

Mejor quinteto encontrado usando búsqueda exhaustiva en Python
Valoration total = 1083.57

            Player Pos  PTS  TRB  AST  STL  BLK  TOV  PF  valoration
Bennedict Mathurin  SF  310   70   25    0    0   25  30     351.933
     Pascal Siakam  PF  241   74   53   11    4   26  33     325.423
   Andrew Nembhard  PG  172   14   68    4    0   22  26     211.579
      Ben Sheppard  SG   68   47   17    4    2    7  26     106.327
          Jay Huff   C   53   31    8    1   19    6  19      88.309

Distribución de posiciones en el quinteto:
Pos
SF    1
PF    1
PG    1
SG    1
C     1
Name: count, dtype: int64
